In [1]:
import pandas as pd
import seaborn as sns
import matplotlib as mat
import xgboost as xg 
from sklearn.model_selection import train_test_split 
from sklearn.metrics import mean_squared_error as MSE 
from sklearn.metrics import r2_score
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import MaxAbsScaler
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error
import altair as alt

housing = pd.read_csv('https://raw.githubusercontent.com/byui-cse/cse450-course/master/data/housing.csv')
housing_nodate = housing[housing.columns.difference(["date"])]

#puts date in datetime object
housing.date = pd.to_datetime(housing.date)#, format="%Y%m%d")
#housing['date']
#creates needed columns

housing['yearBought'] = pd.DatetimeIndex(housing['date']).year
#housing['yearBought']

housing['lastRenovated'] = housing.apply(lambda h: h['yr_renovated'] if h['yr_renovated'] !=0 else h['yr_built'], axis = 1)
#housing['lastRenovated']


housing['timeLastReno'] = housing['date'].dt.year-housing['lastRenovated']
#housing['timeLastReno']
housing.head()

housing["trx_age"] = (housing.date.max() - housing.date).apply(lambda x: x.days)
housing['gradeScaled'] = MinMaxScaler().fit_transform(housing[['grade']])
housing['gradeScaled']
housing['conditionScaled'] = MinMaxScaler().fit_transform(housing[['condition']])
housing['conditionScaled']
housing['viewScaled'] = MinMaxScaler().fit_transform(housing[['view']])
housing['conditionScaled'] = MinMaxScaler().fit_transform(housing[['condition']])

In [2]:
#housing['datenotime'] = housing['date'].dt.striptime('%Y-%m-%d')
housing.head(10)
housing[['sqft_living', 'sqft_lot']].describe()

,sqft_living,sqft_lot
count,21613.000000,2.161300e+04
mean,2079.899736,1.510697e+04
std,918.440897,4.142051e+04
min,290.000000,5.200000e+02
25%,1427.000000,5.040000e+03
50%,1910.000000,7.618000e+03
75%,2550.000000,1.068800e+04
max,13540.000000,1.651359e+06


In [3]:
# No lot = apartment
# over 5000 sqft is a mansion
# large lot (over 1 acre - 43560 sqft)
# size house vs size lot comparison
# average == 2000 sqft
# Small 1000sqft< Medium up to 2250sqft and Large houses 2251 to 5k, mansion 5k+
# Small Medium and Large Acreage(if more than 1 acre of land) Lot
#
# 
# 

In [4]:
housing['gradeScaled'] = MinMaxScaler().fit_transform(housing[['grade']])
housing['gradeScaled']
housing['conditionScaled'] = MinMaxScaler().fit_transform(housing[['condition']])
housing['conditionScaled']
housing['viewScaled'] = MinMaxScaler().fit_transform(housing[['view']])
housing['conditionScaled'] = MinMaxScaler().fit_transform(housing[['condition']])

In [5]:
#housing2 = housing[housing['sqft_living'] < 9000]
housing2 = housing[housing['sqft_living'] < 400000]

In [6]:
housing[['price', 'sqft_living', 'waterfront', 'view']].describe()

,price,sqft_living,waterfront,view
count,2.161300e+04,21613.000000,21613.000000,21613.000000
mean,5.400881e+05,2079.899736,0.007542,0.234303
std,3.671272e+05,918.440897,0.086517,0.766318
min,7.500000e+04,290.000000,0.000000,0.000000
25%,3.219500e+05,1427.000000,0.000000,0.000000
50%,4.500000e+05,1910.000000,0.000000,0.000000
75%,6.450000e+05,2550.000000,0.000000,0.000000
max,7.700000e+06,13540.000000,1.000000,4.000000


In [7]:
#GridSearchCV(housing['price'], X, scoring = mean_squared_error)

In [8]:
#readying for splitting
X = housing.drop(['price', 'date', 'yr_renovated', 'id', 'lat', 'long', 'grade', 'condition'], axis= 1)
y = housing.price


#splitting the data
train_X, test_X, train_y, test_y = train_test_split(X, y, test_size = 0.4)
#train_X, train_y, test_x, test_y = train_test_split(X, y, test_size = 0.3,
#instantiate the model


#xgb_r = xg.XGBRegressor(objective = "reg:linear", n_estimators=15)

train_dmatrix = xg.DMatrix(data = train_X, label = train_y) 
test_dmatrix = xg.DMatrix(data = test_X, label = test_y) 


# Parameter dictionary specifying base learner 
param = {"booster":"gblinear", "objective":"reg:squarederror"} 
  
xgb_r = xg.train(params = param, dtrain = train_dmatrix, num_boost_round = 500) 
pred = xgb_r.predict(test_dmatrix) 
#xgb_r.fit(train_X, train_y) 

#pred = xgb_r.predict(test_X)

print(MSE(test_y, pred)**0.5)
print(r2_score(test_y, pred))
print(xgb_r.best_iteration)


225659.19108022083
0.6367725539329737
499


In [9]:

xgb_regression = xg.XGBRegressor(objective = "reg:squarederror", n_estimators=15)
xgb_regression.fit(train_X, train_y) 

pred = xgb_regression.predict(test_X)
xgb_regression.score(test_X, test_y)

0.5033667425633928

In [10]:
xgb1 = xg.XGBRegressor()
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:squarederror'],
              'learning_rate': [.03, 0.05, .07], #so called `eta` value
              'max_depth': [5, 6, 7],
              'min_child_weight': [4],
              'silent': [1],
              'subsample': [0.7],
              'colsample_bytree': [0.7],
              'n_estimators': [500]}

xgb_grid = GridSearchCV(xgb1,
                        parameters,
                        cv = 2,
                        n_jobs = 5,
                        verbose=True)

xgb_grid.fit(train_X,
         train_y)

print(xgb_grid.best_score_)
print(xgb_grid.best_params_)


Fitting 2 folds for each of 9 candidates, totalling 18 fits


[Parallel(n_jobs=5)]: Using backend LokyBackend with 5 concurrent workers.
[Parallel(n_jobs=5)]: Done  18 out of  18 | elapsed:  1.3min finished


0.8503141870844539
{'colsample_bytree': 0.7, 'learning_rate': 0.07, 'max_depth': 5, 'min_child_weight': 4, 'n_estimators': 500, 'nthread': 4, 'objective': 'reg:squarederror', 'silent': 1, 'subsample': 0.7}


In [11]:
model =  xgb_grid.best_estimator_
y_predict_train = xgb_grid.best_estimator_.predict(train_X)
y_predict_test = xgb_grid.best_estimator_.predict(test_X)

print(r2_score(y_predict_test, test_y))
print(MSE(y_predict_test, test_y)**0.5)
model.feature_importances_

0.8333951323204389
136029.30253076408


array([0.00771226, 0.03453751, 0.10590027, 0.00993729, 0.0103489 ,
       0.29663065, 0.05010501, 0.02810054, 0.01854974, 0.03993722,
       0.03423019, 0.05469739, 0.0137369 , 0.00337532, 0.0124785 ,
       0.00929404, 0.006818  , 0.22837336, 0.015412  , 0.01982496],
      dtype=float32)

In [12]:
feature_dat = pd.DataFrame({
    "values": model.feature_importances_, 
    "features": X.columns
})

graph1 = (alt.Chart(feature_dat.query('values > 0.02'))
    .encode(
        alt.X('values', title = "Weight (%)"), 
        alt.Y('features', sort = "-x", title = "Features"))
    .mark_bar()
    .properties(
        title = {
            "text": "Feature Importance"
        }
    )
)

graph1

alt.Chart(...)